In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
import os
import random

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# 2nd eLU, tanh, 4, 50, 50, 50, 1
## Model
## Train NN & Predict

Import Data (normalized) & Separate to Training and Test

In [2]:
#data = np.loadtxt("ACL nomalization data.txt")
#data2 = np.loadtxt("acl data.txt")
data = np.loadtxt("ACL FTIR data_mean.txt")

X_NormalizedData = data[:,0:4]
X_NormalizedData = X_NormalizedData - 0.5
Y_NormalizedData = data[:,4]

X_TrainingData = data[0:20,0:4]
X_TrainingData = X_TrainingData - 0.5
Y_TrainingData = data[0:20,4]

X_TestData = data[20:,0:4]
X_TestData = X_TestData - 0.5
Y_TestData = data[20:,4]

Sequence = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

#XY_TrainingData = data[0:21,:]
#X_TrainingData = XY_TrainingData[0:21,0:4]
#Y_TrainingData = XY_TrainingData[0:21,4]
#
#XY_TestData = data[21:,:]
#X_TestData = XY_TestData[:,0:4]
#Y_TestData = XY_TestData[:,4]

print(X_TrainingData)
print("------------------------")
print(X_TestData)
print("------------------------")
print(Y_TrainingData)
print("------------------------")
print(Y_TestData)

[[ 0.   0.   0.5  0.5]
 [ 0.  -0.5  0.   0.5]
 [-0.5 -0.5  0.   0. ]
 [-0.5  0.   0.5  0. ]
 [ 0.5  0.   0.   0.5]
 [ 0.   0.  -0.5  0.5]
 [ 0.5 -0.5  0.   0. ]
 [ 0.   0.  -0.5 -0.5]
 [ 0.  -0.5 -0.5  0. ]
 [ 0.   0.   0.5 -0.5]
 [-0.5  0.   0.  -0.5]
 [ 0.5  0.5  0.   0. ]
 [ 0.  -0.5  0.5  0. ]
 [-0.5  0.5  0.   0. ]
 [ 0.5  0.   0.  -0.5]
 [-0.5  0.  -0.5  0. ]
 [ 0.   0.5  0.5  0. ]
 [ 0.   0.5  0.  -0.5]
 [ 0.5  0.   0.5  0. ]
 [ 0.   0.5 -0.5  0. ]]
------------------------
[[ 0.  -0.5  0.  -0.5]
 [ 0.5  0.  -0.5  0. ]
 [-0.5  0.   0.   0.5]
 [ 0.   0.   0.   0. ]
 [ 0.   0.5  0.   0.5]]
------------------------
[-0.21844449 -0.47152439 -0.45735005 -0.1123416  -0.11245536 -0.18036006
 -0.19692817  0.00145492 -0.14520267  0.01178692  0.04523337  0.52847561
 -0.13421128  0.19149351 -0.00787756  0.09143864 -0.16016158  0.30505231
  0.04222738  0.33060721]
------------------------
[-0.08104142  0.20294599  0.18485121  0.07951933  0.2628122 ]


# 1st eLU, tanh, 4, 5, 5, 5, 5, 1
R_square of EntireData
0.491762023498
R_square of TestData
-8.19693580628

## Model

In [3]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=5
n_2ndHiddenUnit=5
n_3rdHiddenUnit=5
n_4thHiddenUnit=5
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [4]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.0591919
9 1000 0.0959228
18 2000 0.0972936
7 3000 0.0957881
0 4000 0.0972038
18 5000 0.0981984
19 6000 0.0958883
5 7000 0.0973792
14 8000 0.0976975
1 9000 0.0975046
1 10000 0.0990406
7 11000 0.100786
8 12000 0.101334
7 13000 0.100444
16 14000 0.102394
1 15000 0.103348
5 16000 0.103701
7 17000 0.104582
9 18000 0.105857
5 19000 0.104066
19 20000 0.105914
5 21000 0.104616
10 22000 0.104949
7 23000 0.104981
19 24000 0.105593
7 25000 0.104459
0 26000 0.10555
18 27000 0.10458
15 28000 0.106922
13 29000 0.106271
3 30000 0.107948
5 31000 0.104551
15 32000 0.106601
19 33000 0.109628
14 34000 0.105702
6 35000 0.105445
1 36000 0.107343
0 37000 0.108003
8 38000 0.106962
3 39000 0.106481
13 40000 0.10712
17 41000 0.105158
3 42000 0.106845
18 43000 0.105547
14 44000 0.109072
11 45000 0.109383
9 46000 0.107513
13 47000 0.108123
7 48000 0.112371
7 49000 0.107268
6 50000 0.11141
11 51000 0.108614
8 52000 0.109432
13 53000 0.117002
4 54000 0.115507
10 55000 0.112323
4 56000 0.114265
3 57000 0.1223

# 2th eLU, tanh, 4, 5, 5, 3, 3, 1
R_square of EntireData
0.0879968649096
R_square of TestData
-15.5296440731

## Model

In [5]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=5
n_2ndHiddenUnit=5
n_3rdHiddenUnit=3
n_4thHiddenUnit=3
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [6]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

15 0 0.0597166
8 1000 0.0953603
2 2000 0.0944426
17 3000 0.0963795
7 4000 0.0963795
4 5000 0.0958485
0 6000 0.097133
12 7000 0.0980876
12 8000 0.0982557
17 9000 0.0975394
9 10000 0.098718
1 11000 0.100373
14 12000 0.102021
15 13000 0.103701
4 14000 0.10416
6 15000 0.103341
4 16000 0.104254
8 17000 0.10419
10 18000 0.104536
14 19000 0.104724
12 20000 0.104065
18 21000 0.105047
19 22000 0.106695
17 23000 0.104891
3 24000 0.106019
9 25000 0.105744
12 26000 0.104032
1 27000 0.105561
15 28000 0.104748
10 29000 0.105825
16 30000 0.105625
18 31000 0.105387
5 32000 0.104354
5 33000 0.105053
9 34000 0.105998
4 35000 0.104409
4 36000 0.106646
13 37000 0.106219
12 38000 0.108553
5 39000 0.110096
16 40000 0.108568
6 41000 0.108332
4 42000 0.114196
7 43000 0.114663
4 44000 0.108781
17 45000 0.111401
14 46000 0.109848
12 47000 0.122036
15 48000 0.110694
13 49000 0.113502
8 50000 0.109766
17 51000 0.111349
4 52000 0.110082
7 53000 0.114402
6 54000 0.111124
17 55000 0.123863
17 56000 0.113634
9 57000 

# 3rd eLU, tanh, 4, 20, 20, 20, 1
R_square of EntireData
0.671702126264
R_square of TestData
-4.94691754594

## Model

In [7]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=20
n_2ndHiddenUnit=20
n_3rdHiddenUnit=20
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [8]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

5 0 0.058412
8 1000 0.100367
13 2000 0.0966509
6 3000 0.0967577
16 4000 0.100647
12 5000 0.0960784
14 6000 0.0952815
3 7000 0.0949181
19 8000 0.0999141
13 9000 0.100355
8 10000 0.101971
8 11000 0.0998259
6 12000 0.105001
8 13000 0.107792
16 14000 0.107174
15 15000 0.103811
13 16000 0.10591
18 17000 0.103655
7 18000 0.107135
0 19000 0.111169
10 20000 0.108073
2 21000 0.108399
5 22000 0.106963
10 23000 0.104092
13 24000 0.115817
1 25000 0.110288
7 26000 0.108349
8 27000 0.117448
18 28000 0.104903
1 29000 0.105404
13 30000 0.113069
1 31000 0.104067
18 32000 0.11286
12 33000 0.106168
12 34000 0.111838
14 35000 0.11139
17 36000 0.11077
12 37000 0.122134
17 38000 0.112531
14 39000 0.110562
0 40000 0.10956
3 41000 0.123868
5 42000 0.117178
2 43000 0.111359
0 44000 0.107219
8 45000 0.108931
10 46000 0.118463
12 47000 0.103889
17 48000 0.111973
1 49000 0.109355
11 50000 0.116494
9 51000 0.123685
6 52000 0.11601
13 53000 0.112595
11 54000 0.118241
19 55000 0.115132
2 56000 0.114646
1 57000 0.112

# 4th eLU, tanh, 4, 10, 20, 30, 1
R_square of EntireData
0.575180176745
R_square of TestData
-6.72425037375

## Model

In [9]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=10
n_2ndHiddenUnit=20
n_3rdHiddenUnit=30
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [10]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

19 0 0.0597475
3 1000 0.0964565
11 2000 0.0940569
0 3000 0.0989742
11 4000 0.0990248
1 5000 0.0988625
1 6000 0.0970362
12 7000 0.0992741
3 8000 0.092746
16 9000 0.098734
7 10000 0.102468
14 11000 0.0995066
6 12000 0.10282
17 13000 0.103724
1 14000 0.104931
10 15000 0.103118
3 16000 0.106145
9 17000 0.102453
10 18000 0.100412
6 19000 0.107461
17 20000 0.105655
6 21000 0.105097
10 22000 0.106502
3 23000 0.105323
5 24000 0.107999
19 25000 0.108825
7 26000 0.111369
0 27000 0.110698
18 28000 0.105986
6 29000 0.107401
10 30000 0.108991
18 31000 0.111791
1 32000 0.105231
14 33000 0.113548
5 34000 0.108347
1 35000 0.120247
16 36000 0.113965
14 37000 0.109611
17 38000 0.116788
4 39000 0.117658
6 40000 0.114389
16 41000 0.116574
12 42000 0.113811
4 43000 0.113042
6 44000 0.112776
5 45000 0.118605
11 46000 0.117271
2 47000 0.115339
16 48000 0.111501
15 49000 0.115365
0 50000 0.119064
19 51000 0.115285
11 52000 0.114666
6 53000 0.115304
16 54000 0.115296
17 55000 0.11529
1 56000 0.114641
12 57000 

# 5th eLU, tanh, 4, 40, 50, 60, 70, 1
R_square of EntireData
0.810615608259
R_square of TestData
-2.40258045515

## Model

In [11]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=40
n_2ndHiddenUnit=50
n_3rdHiddenUnit=60
n_4thHiddenUnit=70
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
Hypothesis = tf.nn.tanh(tf.matmul(L4, W5) + b5)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [12]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

0 0 0.0586837
16 1000 0.0813662
5 2000 0.084448
6 3000 0.077176
5 4000 0.0940011
16 5000 0.09757
2 6000 0.100815
16 7000 0.105982
12 8000 0.0966139
10 9000 0.107551
14 10000 0.109695
15 11000 0.102497
2 12000 0.108934
4 13000 0.112782
5 14000 0.110663
12 15000 0.128403
19 16000 0.107295
10 17000 0.120805
6 18000 0.12446
19 19000 0.118903
16 20000 0.116528
18 21000 0.114924
9 22000 0.110553
4 23000 0.117864
7 24000 0.118129
12 25000 0.131394
15 26000 0.115053
18 27000 0.121583
11 28000 0.115279
11 29000 0.113733
0 30000 0.116131
16 31000 0.116159
3 32000 0.12088
10 33000 0.112699
6 34000 0.114782
19 35000 0.115301
18 36000 0.115749
16 37000 0.117063
19 38000 0.114877
16 39000 0.114496
15 40000 0.115866
15 41000 0.116528
18 42000 0.115217
11 43000 0.114929
6 44000 0.115407
3 45000 0.112576
11 46000 0.115086
17 47000 0.113386
7 48000 0.113178
12 49000 0.113351
11 50000 0.11524
17 51000 0.115451
16 52000 0.116023
17 53000 0.116675
4 54000 0.115337
6 55000 0.105283
10 56000 0.115034
6 57000

# 6th eLU, tanh, 4, 50, 2, 2, 1
R_square of EntireData
0.701180318268
R_square of TestData
-4.3893905412

## Model

In [13]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=2
n_3rdHiddenUnit=2
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [14]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

7 0 0.0594601
9 1000 0.0934064
15 2000 0.0965876
6 3000 0.0946368
17 4000 0.0964658
9 5000 0.0979333
16 6000 0.0973441
8 7000 0.0982568
9 8000 0.0986618
1 9000 0.0981649
4 10000 0.0963043
2 11000 0.0964808
0 12000 0.0986737
18 13000 0.0966697
13 14000 0.096515
5 15000 0.0969437
17 16000 0.0976925
7 17000 0.0995399
11 18000 0.099032
18 19000 0.0976869
5 20000 0.100851
5 21000 0.099556
12 22000 0.10009
1 23000 0.100578
4 24000 0.100958
18 25000 0.102717
17 26000 0.103648
0 27000 0.102469
15 28000 0.101772
3 29000 0.103947
11 30000 0.104661
12 31000 0.105486
9 32000 0.105851
15 33000 0.106912
16 34000 0.108095
13 35000 0.107389
15 36000 0.108063
7 37000 0.110803
4 38000 0.109853
5 39000 0.106883
13 40000 0.110425
2 41000 0.111069
0 42000 0.10688
16 43000 0.112925
18 44000 0.109092
4 45000 0.114666
14 46000 0.110644
3 47000 0.114949
9 48000 0.115098
0 49000 0.112519
16 50000 0.109139
11 51000 0.115148
1 52000 0.113867
2 53000 0.114957
18 54000 0.11408
16 55000 0.111486
19 56000 0.113863
16

# 7th eLU, tanh, 4, 50, 10, 1
R_square of EntireData
0.539880865102
R_square of TestData
-7.36494950223

## Model

In [15]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=10
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
Hypothesis = tf.nn.tanh(tf.matmul(L2, W3) + b3)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [16]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

19 0 0.0577447
4 1000 0.0993924
15 2000 0.0958965
14 3000 0.0998972
11 4000 0.0969414
2 5000 0.0975838
10 6000 0.102921
11 7000 0.101741
15 8000 0.10482
2 9000 0.102175
16 10000 0.104245
13 11000 0.103003
11 12000 0.101066
14 13000 0.102098
11 14000 0.103142
0 15000 0.104334
13 16000 0.100961
8 17000 0.106286
17 18000 0.103284
16 19000 0.107802
4 20000 0.107784
6 21000 0.107092
14 22000 0.110817
3 23000 0.105056
2 24000 0.108719
2 25000 0.110693
2 26000 0.107626
16 27000 0.112719
1 28000 0.114173
12 29000 0.113733
11 30000 0.111946
6 31000 0.117163
13 32000 0.110595
18 33000 0.114331
19 34000 0.114433
7 35000 0.114561
1 36000 0.115473
8 37000 0.110441
7 38000 0.115779
19 39000 0.111459
14 40000 0.113994
2 41000 0.117166
6 42000 0.115462
12 43000 0.11451
19 44000 0.114293
14 45000 0.117195
2 46000 0.114973
3 47000 0.115104
17 48000 0.11252
8 49000 0.115724
18 50000 0.115669
15 51000 0.115218
15 52000 0.112333
16 53000 0.115368
2 54000 0.114829
4 55000 0.116792
1 56000 0.126323
9 57000 0

# 8th eLU 4, 50, 50, 50, 1
R_square of EntireData
0.832872722312
R_square of TestData
-2.03928093358

## Model

In [17]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=50
n_3rdHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.elu(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [18]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

11 0 0.0598591
9 1000 0.0914237
17 2000 0.0979066
16 3000 0.0944706
16 4000 0.103999
15 5000 0.0968753
1 6000 0.102053
0 7000 0.105114
3 8000 0.110146
16 9000 0.115229
11 10000 0.107359
19 11000 0.10457
5 12000 0.111356
14 13000 0.102579
2 14000 0.109531
10 15000 0.113722
8 16000 0.107954
15 17000 0.111239
5 18000 0.110317
15 19000 0.111807
7 20000 0.115354
15 21000 0.112118
1 22000 0.117924
9 23000 0.109133
14 24000 0.119852
12 25000 0.125901
14 26000 0.119969
15 27000 0.113637
8 28000 0.122699
5 29000 0.11928
10 30000 0.109864
11 31000 0.109449
13 32000 0.115359
19 33000 0.118647
9 34000 0.112424
4 35000 0.117118
10 36000 0.113096
9 37000 0.115166
16 38000 0.121802
1 39000 0.115921
12 40000 0.115253
14 41000 0.10848
15 42000 0.121393
1 43000 0.117291
18 44000 0.119663
7 45000 0.114446
10 46000 0.114564
3 47000 0.116915
10 48000 0.115696
8 49000 0.114827
7 50000 0.117232
6 51000 0.112185
2 52000 0.116451
18 53000 0.116935
1 54000 0.118424
2 55000 0.113852
0 56000 0.114224
2 57000 0.11

# 9th eLU, tanh, 4, 50, 30, 50, 1
R_square of EntireData
0.630267524699
R_square of TestData
-5.72368478092

## Model

In [19]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=50
n_2ndHiddenUnit=30
n_3rdHiddenUnit=50
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
Hypothesis = tf.nn.tanh(tf.matmul(L3, W4) + b4)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [20]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder,step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

1 0 0.0578097
8 1000 0.104041
15 2000 0.0983504
2 3000 0.0930979
8 4000 0.0932963
8 5000 0.101727
12 6000 0.105457
19 7000 0.0996321
3 8000 0.101341
4 9000 0.0982755
0 10000 0.103124
15 11000 0.0998071
12 12000 0.108508
15 13000 0.102191
11 14000 0.10747
12 15000 0.112387
16 16000 0.104686
14 17000 0.0961403
11 18000 0.113374
13 19000 0.104729
6 20000 0.123605
17 21000 0.110309
11 22000 0.119569
10 23000 0.110472
19 24000 0.117266
8 25000 0.117665
4 26000 0.116432
0 27000 0.11503
14 28000 0.108145
6 29000 0.119603
1 30000 0.112214
8 31000 0.115803
12 32000 0.106642
17 33000 0.112969
13 34000 0.116522
0 35000 0.111167
1 36000 0.113808
16 37000 0.116378
17 38000 0.115915
1 39000 0.118066
7 40000 0.114333
11 41000 0.116423
19 42000 0.114417
13 43000 0.113679
7 44000 0.115322
11 45000 0.115138
0 46000 0.113851
18 47000 0.115664
15 48000 0.120533
14 49000 0.114185
19 50000 0.115286
1 51000 0.113196
7 52000 0.115491
16 53000 0.115995
9 54000 0.114901
3 55000 0.115417
11 56000 0.116224
1 5700

# 10th eLU, tanh, 4, 10, 5, 10, 5, 10, 5, 1
R_square of EntireData
0.65002034887
R_square of TestData
-5.33083609998

## Model

In [21]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=10
n_2ndHiddenUnit=5
n_3rdHiddenUnit=10
n_4thHiddenUnit=5
n_5thHiddenUnit=10
n_6thHiddenUnit=5
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_3rdHiddenUnit],stddev=0.01))
W4 = tf.Variable(tf.random_normal([n_3rdHiddenUnit, n_4thHiddenUnit],stddev=0.01))
W5 = tf.Variable(tf.random_normal([n_4thHiddenUnit, n_5thHiddenUnit],stddev=0.01))
W6 = tf.Variable(tf.random_normal([n_5thHiddenUnit, n_6thHiddenUnit],stddev=0.01))
W7 = tf.Variable(tf.random_normal([n_6thHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_3rdHiddenUnit], stddev=0.01))
b4 = tf.Variable(tf.random_normal([n_4thHiddenUnit], stddev=0.01))
b5 = tf.Variable(tf.random_normal([n_5thHiddenUnit], stddev=0.01))
b6 = tf.Variable(tf.random_normal([n_6thHiddenUnit], stddev=0.01))
b7 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
L3 = tf.nn.elu(tf.matmul(L2, W3) + b3)
L4 = tf.nn.elu(tf.matmul(L3, W4) + b4)
L5 = tf.nn.elu(tf.matmul(L4, W5) + b5)
L6 = tf.nn.elu(tf.matmul(L5, W6) + b6)
Hypothesis = tf.nn.tanh(tf.matmul(L6, W7) + b7)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [22]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(100000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder, step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

18 0 0.0588626
7 1000 0.05751
5 2000 0.0575098
7 3000 0.0575095
18 4000 0.0876663
14 5000 0.0952687
5 6000 0.0952653
0 7000 0.0941646
15 8000 0.100329
15 9000 0.105696
17 10000 0.103891
5 11000 0.103319
12 12000 0.109899
6 13000 0.106556
3 14000 0.106418
10 15000 0.106722
2 16000 0.10627
5 17000 0.113369
11 18000 0.105539
13 19000 0.109041
2 20000 0.107679
4 21000 0.113016
5 22000 0.109042
18 23000 0.114648
0 24000 0.113606
19 25000 0.111849
12 26000 0.11019
13 27000 0.112587
0 28000 0.109758
11 29000 0.111665
11 30000 0.114578
18 31000 0.109267
19 32000 0.114746
12 33000 0.11678
9 34000 0.11131
16 35000 0.110497
10 36000 0.117077
10 37000 0.115443
5 38000 0.110542
8 39000 0.112324
16 40000 0.115057
18 41000 0.114785
10 42000 0.112093
19 43000 0.109926
5 44000 0.114256
10 45000 0.109574
6 46000 0.114539
10 47000 0.113398
7 48000 0.11171
1 49000 0.116477
9 50000 0.113038
16 51000 0.117898
4 52000 0.114571
15 53000 0.1081
5 54000 0.117119
19 55000 0.118219
9 56000 0.110107
7 57000 0.1209

# 11th eLU, tanh, 4, 100, 100, 1, training 5만
R_square of EntireData
0.680782977573
R_square of TestData
-4.76003266007

## Model

In [23]:
learning_rate = 0.001
tf.set_random_seed(777)

n_InputUnit = 4
n_1stHiddenUnit=100
n_2ndHiddenUnit=100
n_OutputUnit = 1

W1 = tf.Variable(tf.random_normal([n_InputUnit, n_1stHiddenUnit],stddev=0.01))
W2 = tf.Variable(tf.random_normal([n_1stHiddenUnit, n_2ndHiddenUnit],stddev=0.01))
W3 = tf.Variable(tf.random_normal([n_2ndHiddenUnit, n_OutputUnit],stddev=0.01))

b1 = tf.Variable(tf.random_normal([n_1stHiddenUnit], stddev=0.01))
b2 = tf.Variable(tf.random_normal([n_2ndHiddenUnit], stddev=0.01))
b3 = tf.Variable(tf.random_normal([n_OutputUnit], stddev=0.01))

X = tf.placeholder(tf.float32, [None, 4]) # tensorflow의 placeholder라는 변수를 선언
Y = tf.placeholder(tf.float32, [None,])

L1 = tf.nn.elu(tf.matmul(X, W1) + b1)
L2 = tf.nn.elu(tf.matmul(L1, W2) + b2)
Hypothesis = tf.nn.tanh(tf.matmul(L2, W3) + b3)

cost = tf.reduce_mean(tf.square(Y-Hypothesis))
train = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.5).minimize(cost)

## Train NN & Predict

In [29]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    for step in range(50000):
        SGDorder = Sequence[step % 20] # training을 0번부터 20번까지 순차적으로 돌린다.
        session.run(train, feed_dict ={X: X_TrainingData[SGDorder: SGDorder + 1,:], Y: Y_TrainingData[SGDorder: SGDorder + 1]})
        #session.run(train, feed_dict ={X: X_TrainingData, Y: Y_TrainingData})
        
        if step % 1000 == 0:
            print (SGDorder, step, session.run(cost, feed_dict = {X: X_TrainingData, Y: Y_TrainingData}))
            #print (W1)
            #print (b1)
        if step % 20 == 0:
            random.shuffle(Sequence)
            
    Predict_to_TestData = session.run(Hypothesis, feed_dict={X : X_TestData})
    Predict_to_TrainingData = session.run(Hypothesis, feed_dict={X : X_TrainingData})
    Predict_to_EntireData = session.run(Hypothesis, feed_dict = {X : X_NormalizedData})
    
    print("R_square of EntireData")
    print(r2_score(Y_NormalizedData, Predict_to_EntireData))
    print("R_square of TestData")
    print(r2_score(Y_TestData, Predict_to_TestData))
    print("Result of predicting TrainingData")
    print(Predict_to_TrainingData)
    print("Result of predicting TestData")
    print(Predict_to_TestData)
    print("Y_NormalizedData")
    print(Y_NormalizedData)

16 0 0.0591034
10 1000 0.095497
0 2000 0.100748
10 3000 0.10011
19 4000 0.10358
2 5000 0.102973
16 6000 0.107615
17 7000 0.104846
4 8000 0.102327
15 9000 0.10647
3 10000 0.100107
12 11000 0.102766
17 12000 0.100352
17 13000 0.102319
11 14000 0.102009
17 15000 0.108424
4 16000 0.106031
15 17000 0.101045
3 18000 0.106557
1 19000 0.108531
1 20000 0.107142
16 21000 0.108163
5 22000 0.100563
15 23000 0.10887
11 24000 0.113715
11 25000 0.11393
2 26000 0.110144
10 27000 0.121316
1 28000 0.112117
0 29000 0.116818
7 30000 0.113812
16 31000 0.112864
9 32000 0.113096
12 33000 0.121244
1 34000 0.111504
12 35000 0.115022
5 36000 0.116123
0 37000 0.111989
3 38000 0.114672
3 39000 0.114947
18 40000 0.116181
0 41000 0.11369
12 42000 0.114471
4 43000 0.113867
7 44000 0.115502
19 45000 0.113971
0 46000 0.113858
8 47000 0.113345
8 48000 0.115065
12 49000 0.113724
R_square of EntireData
0.680782977573
R_square of TestData
-4.76003266007
Result of predicting TrainingData
[[-0.2106258 ]
 [-0.46098411]
 [-0.